pdf converter and summarizer working togather

In [ ]:

# -*- coding: utf-8 -*-
#install pdfminer first
# PDF to Text Converter function--------------
from pdfminer.pdfinterp import PDFResourceManager,PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import BytesIO
def pdf_to_text(path):
    manager = PDFResourceManager()
    retstr = BytesIO()
    layout = LAParams(all_texts=True)
    device = TextConverter(manager, retstr, laparams=layout)
    filepath = open(path, 'rb')#.read()
    interpreter = PDFPageInterpreter(manager, device)
    
    for page in PDFPage.get_pages(filepath, check_extractable=True):
        interpreter.process_page(page)
        text = retstr.getvalue()
    filepath.close()
    device.close()
    retstr.close()
    return text


#Summarizer Function   -------------
def summarizer(text):
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize,sent_tokenize
    from nltk.stem.snowball import SnowballStemmer
    import nltk
    import requests
    import json
    import re

    #text=request.json.get('text')   
    #text = request.get_json('text')
    #text=request.json.get('text')
    #text= re.sub(r"[[0-9]+]","",text)
    #text=re.sub(r'\([^)]*\)', '', text)
    #text= re.sub(r"\[","",text)

    text=str(text) # CONVERTING IS REQUIRED DO NOT REMOVE
    
    #text=re.replaceAll("(\r?\n)+", "\n",text)
    #text=re.sub("(\\n){2,}", "\n",text)
    #print(text)
    #================================================
    text =re.sub(r"\n\uf0b7 ","\n",text)
    text =re.sub(r'(\n){2,}', r'\n', text)
    text =re.sub(r'(\n ){2,}', r'\n', text)
    #================================================ these where added on 4-oct-19 to remove multiple \n
    
    text= re.sub(r"[\u2022,\u2023,\u25E6,\u2043,\u2219]\s\d\.\s[A-z]","\n", text)
    stemmer = SnowballStemmer("english")
    stopWords = set(stopwords.words("english"))
    words = word_tokenize(text)
    freqTable = dict()
    for word in words:
        word = word.lower()
        if word in stopWords:
            continue
        word = stemmer.stem(word)
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1

    sentences = sent_tokenize(text)
    sentenceValue = dict()

    for sentence in sentences:
        for word, freq in freqTable.items():
            if word in sentence.lower():
                if sentence in sentenceValue:
                    sentenceValue[sentence] += freq
                else:
                    sentenceValue[sentence] = freq
    #print("\n\n",sentenceValue,"\n\n")#=======>
    sumValues = 0
    for sentence in sentenceValue:
        sumValues += sentenceValue[sentence]

    # Average value of a sentence from original text
    average = int(sumValues / len(sentenceValue))

    summary = {}
    key=1
    for sentence in sentences:
        if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.3 * average)):
    #print(sentenceValue[sentence]," with avg= ",1.5 * average)#it was 1.2 initially
            summary[key] = sentence
            key+=1

    clean_10_summary={}
    key_num=1
    for sent in summary.keys():
        if len(summary[sent]) > 20:
            clean_10_summary[key_num]=summary[sent]
            key_num+=1
            #if key_num==21:
                #break     
        else:
            continue
    return clean_10_summary

   # if __name__="apps":
   #     text=open(r"E:\stwork\iso OutPut\ARTICLE.pdf")#its a local pdf 
        #text=pdfconverter().ipynb_checkpoints
        
        #text=pdfminer(DeprecationWarning) 
    if __name__ == "__main__":
        text = pdf_to_text(r'E:\stwork\SOME_RANDOM_ARTICLE.pdf')#its a local pdf 
        #print(text)
        summ=summarizer(str(text, 'utf-8'))
        #print(str(text, 'utf-8'))
        #return summ
        for k in summ.keys():
            print("\n",summ[k],"\n")
    


Thank you